# Лабораторная работа 6

In [ ]:
!pip install pykeen

### Импорт зависимостей

In [ ]:
import torch
import pykeen
import matplotlib.pyplot as plt
from pykeen.pipeline import pipeline
from pykeen.predict import predict_target
from pykeen.evaluation import ClassificationEvaluator

### Создание датасета

In [ ]:
from pykeen.datasets import CoDExSmall

In [ ]:
dataset = CoDExSmall()

### Инициализация метрик

In [ ]:
roc_auc_score = {"ComplEx": [], "HolE": []}
precision = {"ComplEx": [], "HolE": []}
accuracy = {"ComplEx": [], "HolE": []}
f1 = {"ComplEx": [], "HolE": []}

pipeline_results_list = {"ComplEx": [], "HolE": []}

In [ ]:
evaluator = ClassificationEvaluator()

### Обучение модели

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Список значений количества эпох
epochs = [1,20,30]
for model_name in pipeline_results_list.keys():
    for epoch in epochs:

      # Обучаем модель
      pipeline_results = pipeline(
          model=model_name,
          dataset=dataset,
          training_kwargs=dict(num_epochs=epoch),
          device=device
      )

      model = pipeline_results.model
      # Рассчитываем метрики для обученной модели и датасета Nations
      metrics = evaluator.evaluate(model, dataset.testing.mapped_triples,
                                 additional_filter_triples=[dataset.training.mapped_triples,
                                                            dataset.validation.mapped_triples])

      # Сохраняем рассчитанные метрики
      roc_auc_score[model_name].append(metrics.get_metric('roc_auc_score'))
      precision[model_name].append(metrics.get_metric('average_precision_score'))
      accuracy[model_name].append(metrics.get_metric('accuracy_score'))
      f1[model_name].append(metrics.get_metric('f1_score'))
        
      # Получаем доступное отношение и сущность
      available_relation = next(iter(dataset.relation_to_id))
      available_entity = next(iter(dataset.entity_to_id))

        # Используем predict_target с доступным отношением и сущностью
      pred = predict_target(
            model=pipeline_results.model,
            relation=available_relation,
            head=available_entity,
            triples_factory=pipeline_results.training,
        )

      pred_filtered = pred.filter_triples(dataset.training)
      pred_annotated = pred_filtered.add_membership_columns(validation=dataset.validation, testing=dataset.testing)
      pred_df = pred_annotated.df[['tail_label', 'score']]
      print(f'Результат при вычислении {epoch} эпох для сущности "{available_entity}" и отношения "{available_relation}"')
      print(pred_df)
      print('__________________________')
        
      del pipeline_results

### Отображение метрик

In [ ]:
def draw_metrics(roc_auc_score, precision, accuracy, f1, name):
    # В качесте лейблов используем список значений эпох
    labels = epochs

    # Plotting ROC AUC scores
    plt.plot(labels, roc_auc_score[name], label='ROC_AUC')

    # Plotting precision scores
    plt.plot(labels, precision[name], label='Precision')

    # Plotting accuracy scores
    plt.plot(labels, accuracy[name], label='Accuracy')

    # Plotting F1
    plt.plot(labels, f1[name], label='F1')

    # Добавляем легенду, лейблы и название
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('Scores')
    plt.title('Model Evaluation Metrics for ' + name)

    # Отрисовываем график
    plt.show()

In [ ]:
draw_metrics(roc_auc_score, precision, accuracy, f1, "ComplEx")

In [ ]:
draw_metrics(roc_auc_score, precision, accuracy, f1, "HolE")

### Link Prediction

In [ ]:
def print_prediction_results(name):
    print(f'Модель {name}')
    print('---------------------')
    for idx, ep in enumerate(epochs):
        pipeline_result = pipeline_results_list[name][idx]

        # Получаем доступное отношение и сущность
        available_relation = next(iter(dataset.relation_to_id))
        available_entity = next(iter(dataset.entity_to_id))

        # Используем predict_target с доступным отношением и сущностью
        pred = predict_target(
            model=pipeline_result.model,
            relation=available_relation,
            head=available_entity,
            triples_factory=pipeline_result.training,
        )

        pred_filtered = pred.filter_triples(dataset.training)
        pred_annotated = pred_filtered.add_membership_columns(validation=dataset.validation, testing=dataset.testing)
        pred_df = pred_annotated.df[['tail_label', 'score']]
        print(f'Результат при вычислении {ep} эпох для сущности "{available_entity}" и отношения "{available_relation}"')
        print(pred_df)
        print('__________________________')

In [ ]:
print_prediction_results("ComplEx")

In [ ]:
print_prediction_results("HolE")